In [ ]:
"""
ImaraFund Intelligent Matching Engine - COMPLETE VERSION
AI-powered funding matcher with clear, simple recommendations using Gemini 2.5 Flash
"""

import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv

try:
    import google.generativeai as genai
    GEMINI_AVAILABLE = True
except ImportError:
    GEMINI_AVAILABLE = False
    print("⚠️ google-generativeai not installed. Run: pip install google-generativeai")

# Load environment variables
load_dotenv()

class IntelligentMatcher:
    def __init__(self):
        print("🤖 Initializing ImaraFund AI Matcher...")
        
        # Load datasets
        try:
            self.grants = pd.read_csv('data/grants/final_merged_enhanced.csv')
            self.companies = pd.read_csv('data/companies/synthetic_companies.csv')
            print(f"✅ Loaded {len(self.grants)} grants and {len(self.companies)} companies")
        except FileNotFoundError as e:
            print(f"❌ Error loading data: {e}")
            print("💡 Make sure data files exist in data/grants/ and data/companies/")
            raise
        
        # Setup Gemini AI with correct model
        self.ai_enabled = False
        api_key = os.getenv('GEMINI_API_KEY')
        
        if GEMINI_AVAILABLE and api_key:
            try:
                genai.configure(api_key=api_key)
                # FIXED: Using the model you have access to
                self.model = genai.GenerativeModel('gemini-2.5-flash')
                self.ai_enabled = True
                print("✅ Gemini AI enabled - Ready for recommendations!")
            except Exception as e:
                print(f"⚠️ AI setup failed: {e}")
        else:
            print("ℹ️ AI disabled - Add GEMINI_API_KEY to .env file to enable")
        
        # Clean and prepare data
        self._prepare_data()
    
    def _prepare_data(self):
        """Clean and prepare data for matching"""
        # Ensure numeric columns
        if 'estimated_value_amount' in self.grants.columns:
            self.grants['estimated_value_amount'] = pd.to_numeric(
                self.grants['estimated_value_amount'], errors='coerce'
            ).fillna(0)
        else:
            self.grants['estimated_value_amount'] = 0
        
        # Lowercase text columns for matching
        text_columns = ['target_sectors', 'geographic_scope', 'country', 'eligibility_criteria']
        for col in text_columns:
            if col in self.grants.columns:
                self.grants[col] = self.grants[col].astype(str).str.lower()
    
    def find_matches(self, company_profile, top_n=5):
        """
        Find best matching grants for a company profile
        
        Args:
            company_profile: Dict with keys: company_name, sector, nationality, 
                           business_stage, funding_need_usd
            top_n: Number of top matches to return
        
        Returns:
            List of match dictionaries sorted by score
        """
        matches = []
        
        # Calculate match scores for all grants
        for _, grant in self.grants.iterrows():
            score, breakdown = self._calculate_match_score(company_profile, grant)
            
            if score > 30:  # Only include reasonable matches
                matches.append({
                    'program_name': grant.get('program_name', 'Unknown Program'),
                    'institution': grant.get('institution_name', 'Unknown Institution'),
                    'country': grant.get('country', 'Unknown'),
                    'funding_amount': grant.get('estimated_value_amount', 0),
                    'match_score': round(score, 1),
                    'score_breakdown': breakdown,
                    'target_sectors': grant.get('target_sectors', 'General'),
                    'website': grant.get('website_url', 'Not available'),
                    'repayment_required': grant.get('repayment_required', 'Unknown'),
                    'grant_data': grant.to_dict()
                })
        
        # Sort by score and return top N
        matches_sorted = sorted(matches, key=lambda x: x['match_score'], reverse=True)[:top_n]
        
        return matches_sorted
    
    def _calculate_match_score(self, company, grant):
        """
        Calculate comprehensive match score (0-100)
        
        Returns:
            score: Float between 0-100
            breakdown: Dict with score components
        """
        score = 0
        breakdown = {}
        
        # 1. Geographic Match (40 points) - Most important
        geo_score = self._score_geography(company, grant)
        score += geo_score
        breakdown['geographic'] = geo_score
        
        # 2. Sector Match (30 points)
        sector_score = self._score_sector(company, grant)
        score += sector_score
        breakdown['sector'] = sector_score
        
        # 3. Funding Amount Fit (20 points)
        amount_score = self._score_funding_amount(company, grant)
        score += amount_score
        breakdown['amount_fit'] = amount_score
        
        # 4. Stage Bonus (10 points)
        stage_score = self._score_business_stage(company, grant)
        score += stage_score
        breakdown['stage'] = stage_score
        
        return min(100, score), breakdown
    
    def _score_geography(self, company, grant):
        """Score geographic eligibility (0-40 points)"""
        company_country = str(company.get('nationality', '')).lower()
        grant_scope = str(grant.get('geographic_scope', '')).lower()
        grant_country = str(grant.get('country', '')).lower()
        
        # Global programs get full points
        if 'global' in grant_scope:
            return 40
        
        # Exact country match
        if company_country in grant_country or company_country in grant_scope:
            return 40
        
        # Regional matches
        africa_countries = ['nigeria', 'kenya', 'south africa', 'ghana', 'uganda', 'egypt', 
                          'tanzania', 'rwanda', 'ethiopia', 'senegal']
        
        if company_country in africa_countries:
            if 'africa' in grant_scope or 'african' in grant_scope:
                return 35
        
        # No geographic match
        return 0
    
    def _score_sector(self, company, grant):
        """Score sector alignment (0-30 points)"""
        company_sector = str(company.get('sector', '')).lower()
        target_sectors = str(grant.get('target_sectors', '')).lower()
        
        # All sectors accepted
        if 'all' in target_sectors or 'general' in target_sectors or 'any' in target_sectors:
            return 25
        
        # Exact sector match
        if company_sector in target_sectors:
            return 30
        
        # Partial match (e.g., "tech" in "technology")
        sector_words = company_sector.split()
        if any(word in target_sectors for word in sector_words):
            return 20
        
        # No match but not excluded
        return 10
    
    def _score_funding_amount(self, company, grant):
        """Score funding amount fit (0-20 points)"""
        need = company.get('funding_need_usd', 0)
        available = grant.get('estimated_value_amount', 0)
        
        if available == 0 or need == 0:
            return 15  # Unknown amount gets partial credit
        
        ratio = need / available
        
        # Perfect fit: need is 10%-200% of available
        if 0.1 <= ratio <= 2.0:
            return 20
        
        # Good fit: need is 5%-500% of available
        elif 0.05 <= ratio <= 5.0:
            return 15
        
        # Poor fit but not impossible
        else:
            return 8
    
    def _score_business_stage(self, company, grant):
        """Score business stage fit (0-10 points)"""
        stage = str(company.get('business_stage', '')).lower()
        
        # Most grants are flexible on stage
        if stage in ['startup', 'early growth']:
            return 10
        elif stage == 'idea':
            return 8
        elif stage in ['growth', 'scale-up']:
            return 9
        else:
            return 7
    
    def get_ai_recommendation(self, company_profile, match):
        """
        Generate clear, simple AI recommendation using Gemini 2.5 Flash
        
        Args:
            company_profile: Dict with company details
            match: Dict with matched grant details
        
        Returns:
            String with AI recommendation
        """
        if not self.ai_enabled:
            return "🔑 Add your Gemini API key to .env file to get AI-powered recommendations!"
        
        # FIXED: Simple, clear prompt for friendly advice
        prompt = f"""You are a friendly business advisor helping someone who is NOT a finance expert.

COMPANY:
- Business: {company_profile.get('company_name', 'Startup')}
- What they do: {company_profile.get('sector', 'Unknown')}
- Location: {company_profile.get('nationality', 'Unknown')}
- Stage: {company_profile.get('business_stage', 'Unknown')}
- Money needed: ${company_profile.get('funding_need_usd', 0):,}

FUNDING MATCH:
- Program: {match['program_name']}
- Institution: {match['institution']}
- Amount: ${match['funding_amount']:,}
- Match Score: {match['match_score']}/100

Write advice using SIMPLE language that anyone can understand. Include these 4 sections:

**WHY THIS WORKS:**
Explain in 2-3 simple sentences why this funding fits their business. Use everyday words.

**WHAT TO DO NEXT:**
Give 3 specific actions they can take today. Use simple words like "create a budget" not "develop financial projections."

**WATCH OUT FOR:**
Mention 1-2 realistic challenges in plain English. Be honest but encouraging.

**YOUR CHANCES:**
Say "Excellent", "Good", "Fair", or "Challenging" and explain why in one sentence.

Use everyday words. No jargon. Be encouraging but honest. Keep under 200 words."""

        try:
            response = self.model.generate_content(
                prompt,
                generation_config={
                    'temperature': 0.8,
                    'top_p': 0.9,
                    'max_output_tokens': 600,
                }
            )
            return response.text
        
        except Exception as e:
            error_msg = str(e)
            
            if "quota" in error_msg.lower() or "rate" in error_msg.lower():
                return "⚠️ Too many requests. Free tier: 15 requests/minute. Please wait 60 seconds and try again."
            elif "404" in error_msg or "not found" in error_msg.lower():
                return "⚠️ Model not available. Check available models with verify_gemini.py"
            else:
                return f"⚠️ AI temporarily unavailable: {error_msg}"


# Test function
if __name__ == "__main__":
    print("\n" + "="*60)
    print("ImaraFund INTELLIGENT MATCHER - TEST")
    print("="*60 + "\n")
    
    try:
        # Initialize matcher
        matcher = IntelligentMatcher()
        
        # Create test company profile
        test_company = {
            'company_name': 'Test AgriTech',
            'sector': 'Agriculture',
            'nationality': 'Nigeria',
            'business_stage': 'Startup',
            'funding_need_usd': 50000
        }
        
        print(f"🏢 Test Company: {test_company['company_name']}")
        print(f"   Sector: {test_company['sector']}")
        print(f"   Location: {test_company['nationality']}")
        print(f"   Need: ${test_company['funding_need_usd']:,}")
        print()
        
        # Find matches
        matches = matcher.find_matches(test_company, top_n=3)
        
        if matches:
            print(f"✅ Found {len(matches)} matches\n")
            
            # Display top match
            top_match = matches[0]
            print(f"🏆 Top match: {top_match['program_name']} ({top_match['match_score']}/100)")
            print(f"   Institution: {top_match['institution']}")
            print(f"   Country: {top_match['country']}")
            print(f"   Amount: ${top_match['funding_amount']:,}")
            print()
            
            # Get AI recommendation
            print("🤖 AI Recommendation:")
            print("-" * 60)
            ai_advice = matcher.get_ai_recommendation(test_company, top_match)
            print(ai_advice)
            print("-" * 60)
            
        else:
            print("❌ No matches found")
            
    except Exception as e:
        print(f"❌ Error during test: {e}")
        import traceback
        traceback.print_exc()


In [1]:
import pandas as pd

# Your specific file path
file_path = r"D:\D1\WTF\FundCompass\data\grants\final_merged_enhanced.csv"

def audit_csv(path):
    try:
        # 1. Load the data
        df = pd.read_csv(path)
        
        print("="*50)
        print("CSV DATA AUDIT REPORT")
        print("="*50)

        # 2. Basic Dimensions
        rows, cols = df.shape
        print(f"\n[SUMMARY]")
        print(f"Total Rows:    {rows}")
        print(f"Total Columns: {cols}")

        # 3. Column Audit (Names, Types, and Emptiness)
        print(f"\n[COLUMN AUDIT]")
        
        # Calculate missing values
        null_counts = df.isnull().sum()
        null_percentages = (null_counts / rows) * 100
        unique_counts = df.nunique()
        
        # Combine into a clean audit table
        audit_table = pd.DataFrame({
            'Column Name': df.columns,
            'Data Type': df.dtypes.values,
            'Missing (Qty)': null_counts.values,
            'Missing (%)': null_percentages.values.round(2),
            'Unique Values': unique_counts.values
        })
        
        print(audit_table.to_string(index=False))

        # 4. Statistical Summary (Numerical Columns)
        # Includes Mean, Min, Max, and Percentiles
        if not df.select_dtypes(include=['number']).empty:
            print(f"\n[NUMERICAL STATISTICS]")
            print(df.describe().T.round(2))
        else:
            print("\n[NUMERICAL STATISTICS] No numerical columns found.")

        # 5. Categorical Summary (Non-numerical Columns)
        categorical_cols = df.select_dtypes(include=['object'])
        if not categorical_cols.empty:
            print(f"\n[TOP CATEGORICAL VALUES]")
            # Show the most frequent value for the first few categorical columns
            print(df.describe(include=['object']).T)

        print("\n" + "="*50)
        print("Audit Complete.")

    except FileNotFoundError:
        print(f"Error: Could not find the file at {path}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    audit_csv(file_path)

CSV DATA AUDIT REPORT

[SUMMARY]
Total Rows:    103
Total Columns: 63

[COLUMN AUDIT]
             Column Name Data Type  Missing (Qty)  Missing (%)  Unique Values
              program_id    object              0         0.00            103
            program_name    object              0         0.00            102
        institution_name    object              0         0.00             94
                 country    object              0         0.00             71
                  region    object              0         0.00             14
           currency_code    object              0         0.00             57
  estimated_value_amount    object              1         0.97             44
      repayment_required    object              0         0.00              6
            program_type    object             20        19.42              4
          target_sectors    object              0         0.00             59
         duration_months    object              0       

In [2]:
import pandas as pd

# Your specific file path
file_path = r"D:\D1\WTF\FundCompass\data\companies\synthetic_companies.csv"

def audit_csv(path):
    try:
        # 1. Load the data
        df = pd.read_csv(path)
        
        print("="*50)
        print("CSV DATA AUDIT REPORT")
        print("="*50)

        # 2. Basic Dimensions
        rows, cols = df.shape
        print(f"\n[SUMMARY]")
        print(f"Total Rows:    {rows}")
        print(f"Total Columns: {cols}")

        # 3. Column Audit (Names, Types, and Emptiness)
        print(f"\n[COLUMN AUDIT]")
        
        # Calculate missing values
        null_counts = df.isnull().sum()
        null_percentages = (null_counts / rows) * 100
        unique_counts = df.nunique()
        
        # Combine into a clean audit table
        audit_table = pd.DataFrame({
            'Column Name': df.columns,
            'Data Type': df.dtypes.values,
            'Missing (Qty)': null_counts.values,
            'Missing (%)': null_percentages.values.round(2),
            'Unique Values': unique_counts.values
        })
        
        print(audit_table.to_string(index=False))

        # 4. Statistical Summary (Numerical Columns)
        # Includes Mean, Min, Max, and Percentiles
        if not df.select_dtypes(include=['number']).empty:
            print(f"\n[NUMERICAL STATISTICS]")
            print(df.describe().T.round(2))
        else:
            print("\n[NUMERICAL STATISTICS] No numerical columns found.")

        # 5. Categorical Summary (Non-numerical Columns)
        categorical_cols = df.select_dtypes(include=['object'])
        if not categorical_cols.empty:
            print(f"\n[TOP CATEGORICAL VALUES]")
            # Show the most frequent value for the first few categorical columns
            print(df.describe(include=['object']).T)

        print("\n" + "="*50)
        print("Audit Complete.")

    except FileNotFoundError:
        print(f"Error: Could not find the file at {path}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    audit_csv(file_path)

CSV DATA AUDIT REPORT

[SUMMARY]
Total Rows:    50
Total Columns: 16

[COLUMN AUDIT]
           Column Name Data Type  Missing (Qty)  Missing (%)  Unique Values
            company_id    object              0          0.0             50
          company_name    object              0          0.0             50
                sector    object              0          0.0             10
           nationality    object              0          0.0             15
business_registered_in    object              0          0.0             14
           founder_age     int64              0          0.0             25
        founder_gender    object              0          0.0              2
   business_age_months     int64              0          0.0             35
    annual_revenue_usd     int64              0          0.0             10
             employees     int64              0          0.0             18
        business_stage    object              0          0.0              5
   

In [ ]:
"""
ImaraFund Project Setup & Advanced Data Cleaning
=================================================
This script:
1. Creates proper FastAPI folder structure
2. Archives old files safely (including Control_V2.ipynb)
3. Fixes grants dataset (fills application_url from website_url)
4. Performs comprehensive data cleaning for all 63 columns
5. Creates .gitignore and project files
6. Prepares data for seamless backend integration
"""

import os
import shutil
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import json
import re


class ImaraFundSetup:
    def __init__(self, project_root: str = r"D:\D1\WTF\ImaraFund"):
        self.root = Path(project_root)
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        print("=" * 80)
        print("🚀 ImaraFund Project Setup & Advanced Data Cleaning")
        print("=" * 80)
        print(f"📁 Project Root: {self.root}")
        print(f"⏰ Timestamp: {self.timestamp}\n")
    
    def create_folder_structure(self):
        """Create comprehensive FastAPI project structure."""
        print("📦 Creating ImaraFund folder structure...")
        
        folders = [
            "archive",
            "app",
            "app/api", 
            "app/services",
            "app/core",
            "data",
            "data/grants",
            "data/companies", 
            "data/cleaned",
            "migrations",
            "tests",
            "docs",
            "static"
        ]
        
        for folder in folders:
            folder_path = self.root / folder
            folder_path.mkdir(parents=True, exist_ok=True)
            print(f"  ✓ Created: {folder}")
        
        # Create __init__.py files for Python packages
        init_files = [
            "app/__init__.py",
            "app/api/__init__.py", 
            "app/services/__init__.py",
            "app/core/__init__.py"
        ]
        
        for init_file in init_files:
            init_path = self.root / init_file
            init_path.touch(exist_ok=True)
        
        print("✅ Folder structure created!\n")
    
    def archive_old_files(self):
        """
        Safely move old files and specific notebooks to the archive.
        This includes Control_V2.ipynb and any other legacy files.
        """
        print("📦 Archiving old files and notebooks...")
        
        archive_folder = self.root / "archive" / self.timestamp
        archive_folder.mkdir(parents=True, exist_ok=True)
        
        # Files that MUST STAY in the root (do not move these)
        preserve = {
            "setup_imarafund.py", "app", "data", "migrations", "archive", 
            ".git", ".env", ".gitignore", "requirements.txt", "README.md",
            "venv", ".venv", "ENV"  # Preserve virtual environments
        }
        
        archived_count = 0
        special_files_archived = []  # Track important files like notebooks
        
        for item in self.root.iterdir():
            # If it's NOT in the preserve list, move it to archive
            if item.name not in preserve:
                try:
                    dest = archive_folder / item.name
                    
                    # Move the file or directory
                    if item.is_file():
                        shutil.move(str(item), str(dest))
                    elif item.is_dir():
                        shutil.move(str(item), str(dest))
                    
                    # Special tracking for notebooks and important files
                    if item.suffix in ['.ipynb', '.py', '.csv', '.json']:
                        special_files_archived.append(item.name)
                        print(f"  ✓ Archived (Important): {item.name}")
                    else:
                        print(f"  ✓ Archived: {item.name}")
                    
                    archived_count += 1
                    
                except Exception as e:
                    print(f"  ⚠ Could not archive {item.name}: {e}")
        
        # **SPECIAL CONFIRMATION for Control_V2.ipynb**
        if "Control_V2.ipynb" in special_files_archived:
            print(f"\n  🎯 SPECIAL CONFIRMATION: Control_V2.ipynb successfully archived!")
            print(f"     📍 Location: archive/{self.timestamp}/Control_V2.ipynb")
            print(f"     ✨ Your notebook is safe and can be restored anytime!")
        
        # Summary of special files
        if special_files_archived:
            print(f"\n  📋 Important files archived ({len(special_files_archived)}):")
            for file in special_files_archived:
                print(f"     • {file}")
        
        print(f"\n✅ Archived {archived_count} items to archive/{self.timestamp}\n")
    
    def find_grants_csv(self):
        """Locate the grants CSV file in various possible locations."""
        possible_paths = [
            self.root / "data" / "grants" / "final_merged_enhanced.csv",
            self.root / "final_merged_enhanced.csv",
            self.root / "grants.csv"
        ]
        
        # Search in archive directories as fallback
        archive_base = self.root / "archive"
        if archive_base.exists():
            for archive_dir in sorted(archive_base.glob("*"), reverse=True):
                if archive_dir.is_dir():
                    possible_archive_path = archive_dir / "final_merged_enhanced.csv"
                    if possible_archive_path.exists():
                        return possible_archive_path
        
        for path in possible_paths:
            if path.exists():
                return path
        
        return None
    
    def clean_currency_value(self, value):
        """Clean currency values removing symbols and converting to float."""
        if pd.isna(value) or value == "" or value is None:
            return None
        
        try:
            cleaned = str(value).replace("$", "").replace(",", "").replace(" ", "").strip()
            if not cleaned or cleaned.lower() in ['nan', 'null', 'none', '']:
                return None
            return float(cleaned)
        except (ValueError, TypeError):
            return None
    
    def standardize_boolean(self, value):
        """Convert various boolean representations to proper boolean."""
        if pd.isna(value) or value == "" or value is None:
            return False
        
        if isinstance(value, bool):
            return value
        
        str_val = str(value).lower().strip()
        return str_val in ["true", "yes", "1", "y", "on", "t"]
    
    def clean_text_field(self, value, max_length=None):
        """Clean text fields with length limits."""
        if pd.isna(value) or value is None:
            return None
        
        result = str(value).strip()
        if not result or result.lower() in ['nan', 'null', 'none']:
            return None
        
        if max_length and len(result) > max_length:
            result = result[:max_length-3] + "..."
        
        return result
    
    def fix_grants_dataset(self):
        """Comprehensive grants dataset cleaning and fixing."""
        print("🔧 Fixing and cleaning grants dataset...")
        
        grants_file = self.find_grants_csv()
        if not grants_file:
            print("  ⚠ Grants CSV file not found. Please place 'final_merged_enhanced.csv' in:")
            print("     • Project root: D:\\D1\\WTF\\ImaraFund\\")
            print("     • Or in: data/grants/")
            return None
        
        print(f"  📄 Found grants file: {grants_file}")
        
        # Load dataset with encoding fallback
        try:
            df = pd.read_csv(grants_file, encoding='utf-8')
            print(f"  ✓ Loaded with UTF-8 encoding")
        except UnicodeDecodeError:
            try:
                df = pd.read_csv(grants_file, encoding='latin-1')
                print(f"  ✓ Loaded with Latin-1 encoding")
            except Exception as e:
                print(f"  ❌ Could not read CSV file: {e}")
                return None
        
        print(f"  📊 Loaded {len(df)} grants with {len(df.columns)} columns")
        
        # **CRITICAL FIX: Application URL Column**
        print("\n  🔗 Fixing application URL column...")
        
        # Find the correct column names (case-insensitive)
        app_url_col = None
        website_col = None
        
        for col in df.columns:
            col_lower = col.lower()
            if 'application' in col_lower and ('url' in col_lower or 'link' in col_lower):
                app_url_col = col
                print(f"    ℹ Found application URL column: {col}")
            elif 'website' in col_lower and ('url' in col_lower or 'link' in col_lower):
                website_col = col
                print(f"    ℹ Found website URL column: {col}")
        
        # Create application_url if it doesn't exist
        if not app_url_col:
            app_url_col = 'application_url'
            df[app_url_col] = None
            print(f"    ℹ Created new column: {app_url_col}")
        
        # Count missing values before fix
        missing_before = df[app_url_col].isna().sum()
        print(f"    📉 Missing {app_url_col} before fix: {missing_before} ({missing_before/len(df)*100:.1f}%)")
        
        if website_col and website_col in df.columns:
            missing_website = df[website_col].isna().sum()
            print(f"    📉 Missing {website_col}: {missing_website} ({missing_website/len(df)*100:.1f}%)")
            
            # Use pandas combine_first for elegant filling
            df[app_url_col] = df[app_url_col].combine_first(df[website_col])
            
            missing_after = df[app_url_col].isna().sum()
            filled_count = missing_before - missing_after
            print(f"    ✅ Filled {filled_count} missing {app_url_col} values from {website_col}")
            print(f"    ✅ Missing {app_url_col} after fix: {missing_after} ({missing_after/len(df)*100:.1f}%)")
        else:
            print(f"    ⚠ No website URL column found to fill from")
        
        # **COMPREHENSIVE DATA CLEANING**
        print("\n  🧹 Performing comprehensive data cleaning...")
        
        # 1. Clean currency/numeric columns
        currency_columns = [
            'estimated_value_amount', 'minimum_amount', 'maximum_amount', 
            'minimum_revenue', 'maximum_revenue', 'success_rate'
        ]
        
        cleaned_currency = 0
        for col in currency_columns:
            if col in df.columns:
                df[col] = df[col].apply(self.clean_currency_value)
                cleaned_currency += 1
        
        print(f"    ✓ Cleaned {cleaned_currency} currency columns")
        
        # 2. Standardize boolean columns
        boolean_columns = [
            'repayment_required', 'environmental_focus', 'innovation_focus',
            'digital_focus', 'export_focus', 'women_focused', 'youth_focused',
            'agriculture_focused', 'green_climate_focused', 'technical_assistance',
            'mentorship_available', 'networking_opportunities', 'training_provided',
            'co_financing_required', 'co_financing_available', 'export_support',
            'technology_innovation', 'digital_application', 'verified'
        ]
        
        cleaned_bools = 0
        for col in boolean_columns:
            if col in df.columns:
                df[col] = df[col].apply(self.standardize_boolean)
                cleaned_bools += 1
        
        print(f"    ✓ Standardized {cleaned_bools} boolean columns")
        
        # 3. Clean text fields with length limits
        text_fields = {
            'program_name': 500,
            'institution_name': 500, 
            'eligibility_criteria': None,
            'application_process': None,
            'notes': None,
            'special_features': None,
            'program_description': None
        }
        
        cleaned_text = 0
        for col, max_len in text_fields.items():
            if col in df.columns:
                df[col] = df[col].apply(lambda x: self.clean_text_field(x, max_len))
                cleaned_text += 1
        
        print(f"    ✓ Cleaned {cleaned_text} text columns")
        
        # 4. Handle integer columns
        int_columns = ['duration_months', 'minimum_employees', 'maximum_employees', 
                      'grace_period_months', 'total_beneficiaries', 'year_established']
        
        cleaned_ints = 0
        for col in int_columns:
            if col in df.columns:
                df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
                cleaned_ints += 1
        
        print(f"    ✓ Cleaned {cleaned_ints} integer columns")
        
        # **SAVE CLEANED DATASET**
        output_path = self.root / "data" / "grants" / "final_merged_enhanced_clean.csv"
        output_path.parent.mkdir(parents=True, exist_ok=True)
        
        df.to_csv(output_path, index=False, encoding='utf-8')
        print(f"\n  💾 Cleaned dataset saved to: {output_path.relative_to(self.root)}")
        
        # Generate comprehensive data quality report
        report_path = output_path.parent / "data_quality_report.txt"
        self.generate_data_report(df, report_path)
        
        # Create a backup of original if different location
        if grants_file != output_path:
            backup_path = self.root / "archive" / self.timestamp / grants_file.name
            backup_path.parent.mkdir(parents=True, exist_ok=True)
            shutil.copy2(str(grants_file), str(backup_path))
            print(f"  💾 Original backed up to: archive/{self.timestamp}/{grants_file.name}")
        
        print("✅ Grants dataset cleaning completed!\n")
        return df
    
    def generate_data_report(self, df, output_path):
        """Generate detailed data quality report."""
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("IMARAFUND GRANTS - DATA QUALITY REPORT\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
            f.write("=" * 80 + "\n\n")
            
            f.write(f"📊 DATASET OVERVIEW\n")
            f.write(f"Total Records: {len(df):,}\n")
            f.write(f"Total Columns: {len(df.columns)}\n\n")
            
            f.write("🔍 MISSING DATA ANALYSIS\n")
            f.write("-" * 80 + "\n")
            missing = df.isna().sum()
            missing_pct = (missing / len(df) * 100).round(2)
            
            # Sort by missing percentage (descending)
            missing_sorted = missing_pct.sort_values(ascending=False)
            
            for col in missing_sorted.index:
                if missing[col] > 0:
                    f.write(f"  {col:<35}: {missing[col]:>4} ({missing_pct[col]:>6.1f}%)\n")
            
            f.write(f"\n📈 DATA COMPLETENESS SUMMARY\n")
            f.write("-" * 80 + "\n")
            f.write(f"Columns with 0% missing:    {(missing == 0).sum():>3}\n")
            f.write(f"Columns with <10% missing:  {(missing_pct < 10).sum():>3}\n")
            f.write(f"Columns with 10-50% missing: {((missing_pct >= 10) & (missing_pct < 50)).sum():>3}\n")
            f.write(f"Columns with >50% missing:  {(missing_pct >= 50).sum():>3}\n")
            
            # Key columns for matching algorithm
            f.write(f"\n🎯 KEY MATCHING COLUMNS STATUS\n")
            f.write("-" * 80 + "\n")
            key_columns = {
                'country': 'Geography (40% weight)',
                'geographic_scope': 'Geography (40% weight)',
                'target_sectors': 'Sector (30% weight)',
                'estimated_value_amount': 'Funding (20% weight)',
                'program_name': 'Core identification',
                'institution_name': 'Core identification',
                'application_url': 'Application process (FIXED)',
                'website_url': 'Contact information'
            }
            
            for col, description in key_columns.items():
                if col in df.columns:
                    miss = missing[col]
                    miss_pct = missing_pct[col]
                    status = "✅ Excellent" if miss_pct < 5 else "✓ Good" if miss_pct < 20 else "⚠ Needs attention"
                    f.write(f"  {col:<25}: {status:<20} {miss:>3} missing ({miss_pct:>5.1f}%) - {description}\n")
            
            f.write("\n" + "=" * 80 + "\n")
        
        print(f"  📊 Data quality report saved: {output_path.relative_to(self.root)}")
    
    def create_project_files(self):
        """Create essential project files."""
        print("📝 Creating project configuration files...")
        
        # .gitignore
        gitignore_content = """# ImaraFund .gitignore

# Archive folder - contains old/backup files  
archive/

# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg

# Virtual Environment
venv/
ENV/
env/
.venv/

# IDEs
.vscode/
.idea/
*.swp
*.swo
*~

# Jupyter Notebooks (archived versions)
.ipynb_checkpoints/
*-checkpoint.ipynb

# Environment variables
.env
.env.local
.env.*.local

# Database
*.db
*.sqlite
*.sqlite3
imarafund.db

# Logs
*.log
logs/

# OS
.DS_Store
Thumbs.db

# Testing
.pytest_cache/
.coverage
htmlcov/
.tox/

# Temporary files
*.tmp
*.temp
~$*
"""
        
        with open(self.root / ".gitignore", 'w', encoding='utf-8') as f:
            f.write(gitignore_content)
        print("  ✓ Created: .gitignore")
        
        # requirements.txt
        requirements = """# ImaraFund Backend Requirements

# FastAPI Framework
fastapi==0.109.0
uvicorn[standard]==0.27.0

# Database
sqlalchemy==2.0.25
alembic==1.13.1

# Data Validation  
pydantic==2.5.3
pydantic-settings==2.1.0

# Environment
python-dotenv==1.0.0

# AI Integration (Your Gemini 2.5 Flash)
google-generativeai==0.3.2

# Database Drivers
psycopg2-binary==2.9.9
aiosqlite==0.19.0

# Data Processing (Your IntelligentMatcher dependencies)
pandas==2.1.4
numpy==1.26.3

# API Utilities
python-multipart==0.0.6

# Testing
pytest==7.4.3
httpx==0.25.2

# Development
black==23.12.1
"""
        
        with open(self.root / "requirements.txt", 'w', encoding='utf-8') as f:
            f.write(requirements)
        print("  ✓ Created: requirements.txt")
        
        # .env.example
        env_example = """# ImaraFund Environment Configuration

# Database
DATABASE_URL=sqlite:///./imarafund.db

# API Configuration  
PROJECT_NAME=ImaraFund
API_V1_PREFIX=/api/v1
DEBUG=True

# AI Configuration - Get your key from: https://makersuite.google.com/app/apikey
GEMINI_API_KEY=your_gemini_api_key_here

# Matching Algorithm Weights (Your proven 40/30/20/10 system)
GEOGRAPHY_WEIGHT=0.40
SECTOR_WEIGHT=0.30
FUNDING_WEIGHT=0.20
STAGE_WEIGHT=0.10
"""
        
        with open(self.root / ".env.example", 'w', encoding='utf-8') as f:
            f.write(env_example)
        print("  ✓ Created: .env.example")
        
        print("✅ Project files created!\n")
    
    def run_setup(self):
        """Execute complete setup process."""
        print("🎯 Starting ImaraFund complete setup...\n")
        
        # Step 1: Create folder structure
        self.create_folder_structure()
        
        # Step 2: Create project files
        self.create_project_files()
        
        # Step 3: Archive old files (with confirmation)
        print("=" * 80)
        print("ARCHIVE CONFIRMATION")
        print("=" * 80)
        print("The following will be moved to archive/:")
        print("  • Control_V2.ipynb (your notebook)")
        print("  • Any other files not in the new structure")
        print("  • Old CSV files and scripts")
        print("\nThese files will be SAFE in archive/ and can be restored anytime.")
        print("=" * 80)
        
        response = input("\n📦 Proceed with archiving? (y/n): ").lower()
        if response == 'y':
            self.archive_old_files()
        else:
            print("⏭️ Skipping archive step. You can run this script again later.\n")
        
        # Step 4: Fix and clean grants dataset
        print("=" * 80)
        print("DATA CLEANING")
        print("=" * 80)
        df = self.fix_grants_dataset()
        
        # Final summary
        print("=" * 80)
        print("✅ IMARAFUND SETUP COMPLETE!")
        print("=" * 80)
        print("\n📋 What was accomplished:")
        print("  ✓ Created professional FastAPI folder structure")
        print("  ✓ Generated .gitignore, requirements.txt, .env.example")
        if response == 'y':
            print("  ✓ Safely archived old files (including Control_V2.ipynb)")
        print("  ✓ Fixed grants dataset (filled application_url from website_url)")
        print("  ✓ Performed comprehensive data cleaning")
        print("  ✓ Generated detailed data quality report")
        
        print("\n📋 Next Steps:")
        print("  1. Review cleaned dataset: data/grants/final_merged_enhanced_clean.csv")
        print("  2. Review data quality: data/grants/data_quality_report.txt")
        print("  3. Copy companies CSV to: data/companies/synthetic_companies.csv")
        print("  4. Copy .env.example to .env and add your GEMINI_API_KEY")
        print("  5. Install dependencies: pip install -r requirements.txt")
        print("  6. Run migration: python migrations/migration_script.py")
        print("  7. Start API: uvicorn app.main:app --reload")
        print("  8. Visit: http://localhost:8000/docs")
        
        if response == 'y':
            print("\n💡 Your original work is safely stored in:")
            print(f"   archive/{self.timestamp}/Control_V2.ipynb")
        
        print("\n" + "=" * 80 + "\n")


if __name__ == "__main__":
    setup = ImaraFundSetup()
    setup.run_setup()


In [3]:
"""
ImaraFund Project Structure Setup - Part 1 Only
===============================================
This script ONLY:
1. Creates FastAPI folder structure
2. Archives old files (preserves Control_V2.ipynb)
3. Creates .gitignore file
4. Creates essential project files (requirements.txt, .env.example, README.md)

NO data cleaning. NO backend code. Just the skeleton structure.
"""

import os
import shutil
from pathlib import Path
from datetime import datetime


class ImaraFundStructureSetup:
    def __init__(self, project_root: str = r"D:\D1\WTF\ImaraFund"):
        self.root = Path(project_root)
        self.timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        print("=" * 70)
        print("🚀 ImaraFund Project Structure Setup (Part 1)")
        print("=" * 70)
        print(f"📁 Project Root: {self.root}")
        print(f"⏰ Timestamp: {self.timestamp}\n")
    
    def create_folder_structure(self):
        """Create FastAPI project folder structure."""
        print("📦 Creating ImaraFund folder structure...")
        
        folders = [
            "archive",
            "app",
            "app/api", 
            "app/services",
            "app/core",
            "data",
            "data/grants",
            "data/companies", 
            "data/cleaned",
            "migrations",
            "tests",
            "docs",
            "static"
        ]
        
        for folder in folders:
            folder_path = self.root / folder
            folder_path.mkdir(parents=True, exist_ok=True)
            print(f"  ✓ Created: {folder}/")
        
        # Create __init__.py files for Python packages
        init_files = [
            "app/__init__.py",
            "app/api/__init__.py", 
            "app/services/__init__.py",
            "app/core/__init__.py"
        ]
        
        for init_file in init_files:
            init_path = self.root / init_file
            init_path.touch(exist_ok=True)
        
        print("✅ Folder structure created!\n")
    
    def create_gitignore(self):
        """Create comprehensive .gitignore file."""
        print("📝 Creating .gitignore...")
        
        gitignore_content = """# ImaraFund .gitignore

# Archive folder - contains old/backup files  
archive/

# Python
__pycache__/
*.py[cod]
*$py.class
*.so
.Python
build/
develop-eggs/
dist/
downloads/
eggs/
.eggs/
lib/
lib64/
parts/
sdist/
var/
wheels/
*.egg-info/
.installed.cfg
*.egg

# Virtual Environment
venv/
ENV/
env/
.venv/

# IDEs
.vscode/
.idea/
*.swp
*.swo
*~

# Jupyter Notebooks
.ipynb_checkpoints/
*-checkpoint.ipynb

# Environment variables
.env
.env.local
.env.*.local

# Database
*.db
*.sqlite
*.sqlite3
imarafund.db

# Logs
*.log
logs/

# OS
.DS_Store
Thumbs.db

# Testing
.pytest_cache/
.coverage
htmlcov/
.tox/

# Temporary files
*.tmp
*.temp
~$*
"""
        
        gitignore_path = self.root / ".gitignore"
        with open(gitignore_path, 'w', encoding='utf-8') as f:
            f.write(gitignore_content)
        
        print(f"  ✓ Created: .gitignore")
        print("✅ .gitignore created!\n")
    
    def create_project_files(self):
        """Create essential project files."""
        print("📝 Creating project files...")
        
        # requirements.txt
        requirements = """# ImaraFund Backend Requirements

# FastAPI Framework
fastapi==0.109.0
uvicorn[standard]==0.27.0

# Database
sqlalchemy==2.0.25

# Data Validation  
pydantic==2.5.3
pydantic-settings==2.1.0

# Environment
python-dotenv==1.0.0

# AI Integration (Gemini 2.5 Flash)
google-generativeai==0.3.2

# Database Drivers
psycopg2-binary==2.9.9

# Data Processing
pandas==2.1.4
numpy==1.26.3

# API Utilities
python-multipart==0.0.6
"""
        
        with open(self.root / "requirements.txt", 'w', encoding='utf-8') as f:
            f.write(requirements)
        print("  ✓ Created: requirements.txt")
        
        # .env.example
        env_example = """# ImaraFund Environment Configuration

# Database
DATABASE_URL=sqlite:///./imarafund.db

# API Configuration  
PROJECT_NAME=ImaraFund
API_V1_PREFIX=/api/v1
DEBUG=True

# AI Configuration
GEMINI_API_KEY=your_gemini_api_key_here

# Matching Algorithm Weights
GEOGRAPHY_WEIGHT=0.40
SECTOR_WEIGHT=0.30
FUNDING_WEIGHT=0.20
STAGE_WEIGHT=0.10
"""
        
        with open(self.root / ".env.example", 'w', encoding='utf-8') as f:
            f.write(env_example)
        print("  ✓ Created: .env.example")
        
        # README.md
        readme = """# ImaraFund

AI-powered funding matcher for African SMEs.

## Project Structure

Copy
ImaraFund/ ├── app/ # FastAPI application │ ├── api/ # API endpoints │ ├── services/ # Business logic (matching, AI) │ └── core/ # Configuration ├── data/ # Data files │ ├── grants/ # Grant datasets │ ├── companies/ # Company datasets │ └── cleaned/ # Cleaned data outputs ├── migrations/ # Database migration scripts ├── tests/ # Unit tests ├── docs/ # Documentation └── static/ # Static files


## Next Steps

1. Add backend code to `app/` folder
2. Place CSV files in `data/grants/` and `data/companies/`
3. Create migration scripts in `migrations/`
4. Install dependencies: `pip install -r requirements.txt`

## Archive

Old files are moved to `archive/` with timestamps for safety.
"""
        
        with open(self.root / "README.md", 'w', encoding='utf-8') as f:
            f.write(readme)
        print("  ✓ Created: README.md")
        
        print("✅ Project files created!\n")
    
    def archive_old_files(self):
        """Archive old files while preserving Control_V2.ipynb and new structure."""
        print("📦 Archiving old files...")
        print("  ℹ️  Preserving Control_V2.ipynb (you're working from it)")
        
        archive_folder = self.root / "archive" / self.timestamp
        archive_folder.mkdir(parents=True, exist_ok=True)
        
        # Files that MUST STAY in the root
        preserve = {
            "Control_V2.ipynb",       # Your working notebook
            "setup_imarafund.py",     # This script
            "app", "data", "migrations", "archive", "tests", "docs", "static",  # New structure
            ".git", ".gitignore", ".env",  # Git and config
            "requirements.txt", ".env.example", "README.md",  # Project files
            "venv", ".venv", "ENV", "env"  # Virtual environments
        }
        
        archived_count = 0
        important_files = []
        
        for item in self.root.iterdir():
            if item.name in preserve:
                continue
            
            try:
                dest = archive_folder / item.name
                
                if item.is_file():
                    shutil.move(str(item), str(dest))
                elif item.is_dir():
                    shutil.move(str(item), str(dest))
                
                # Track important files
                if item.suffix in ['.ipynb', '.py', '.csv', '.json', '.xlsx']:
                    important_files.append(item.name)
                    print(f"  ✓ Archived (Important): {item.name}")
                else:
                    print(f"  ✓ Archived: {item.name}")
                
                archived_count += 1
                
            except Exception as e:
                print(f"  ⚠ Could not archive {item.name}: {e}")
        
        if important_files:
            print(f"\n  📋 Important files safely archived:")
            for file in important_files:
                print(f"     • {file}")
        
        print(f"\n✅ Archived {archived_count} items to archive/{self.timestamp}")
        print("✅ Control_V2.ipynb preserved in root\n")
    
    def run_setup(self):
        """Execute the complete structure setup."""
        print("🎯 Starting structure-only setup...\n")
        
        # Step 1: Create folder structure
        self.create_folder_structure()
        
        # Step 2: Create .gitignore
        self.create_gitignore()
        
        # Step 3: Create project files
        self.create_project_files()
        
        # Step 4: Archive confirmation
        print("=" * 70)
        print("ARCHIVE CONFIRMATION")
        print("=" * 70)
        print("Archive old files while keeping Control_V2.ipynb in root?")
        print("(Your notebook will be safe - only other files will be moved)")
        print("=" * 70)
        
        response = input("\nProceed with archiving? (y/n): ").lower().strip()
        
        if response == 'y':
            self.archive_old_files()
        else:
            print("⏭️ Skipping archive step.\n")
        
        # Final summary
        print("=" * 70)
        print("✅ IMARAFUND STRUCTURE SETUP COMPLETE!")
        print("=" * 70)
        print("\n📋 Created:")
        print("  ✓ FastAPI folder structure")
        print("  ✓ .gitignore (archive/ ignored)")
        print("  ✓ requirements.txt")
        print("  ✓ .env.example")
        print("  ✓ README.md")
        
        if response == 'y':
            print(f"  ✓ Old files archived to: archive/{self.timestamp}/")
            print("  ✓ Control_V2.ipynb kept in root")
        
        print("\n📂 Your clean structure:")
        print("  ImaraFund/")
        print("  ├── Control_V2.ipynb     (your notebook)")
        print("  ├── app/                 (backend code goes here)")
        print("  ├── data/                (CSV files go here)")
        print("  ├── migrations/          (migration scripts)")
        print("  ├── archive/             (old files, git-ignored)")
        print("  ├── .gitignore")
        print("  ├── requirements.txt")
        print("  └── README.md")
        
        print("\n📋 Next Steps:")
        print("  1. Continue working from Control_V2.ipynb")
        print("  2. When ready, request the 10-step backend implementation")
        print("  3. Copy backend files into the app/ structure")
        print("  4. Place your CSV files in data/grants/ and data/companies/")
        
        print("\n" + "=" * 70 + "\n")


# Run the setup
if __name__ == "__main__":
    setup = ImaraFundStructureSetup()
    setup.run_setup()



🚀 ImaraFund Project Structure Setup (Part 1)
📁 Project Root: D:\D1\WTF\ImaraFund
⏰ Timestamp: 20260219_200425

🎯 Starting structure-only setup...

📦 Creating ImaraFund folder structure...
  ✓ Created: archive/
  ✓ Created: app/
  ✓ Created: app/api/
  ✓ Created: app/services/
  ✓ Created: app/core/
  ✓ Created: data/
  ✓ Created: data/grants/
  ✓ Created: data/companies/
  ✓ Created: data/cleaned/
  ✓ Created: migrations/
  ✓ Created: tests/
  ✓ Created: docs/
  ✓ Created: static/
✅ Folder structure created!

📝 Creating .gitignore...
  ✓ Created: .gitignore
✅ .gitignore created!

📝 Creating project files...
  ✓ Created: requirements.txt
  ✓ Created: .env.example
  ✓ Created: README.md
✅ Project files created!

ARCHIVE CONFIRMATION
Archive old files while keeping Control_V2.ipynb in root?
(Your notebook will be safe - only other files will be moved)
📦 Archiving old files...
  ℹ️  Preserving Control_V2.ipynb (you're working from it)
  ✓ Archived: .qodo
  ✓ Archived (Important): app.py
  

In [4]:
"""
ImaraFund Grants Data Cleaning Script
====================================
Primary fix: Fill data_source_url from website_url (83 missing → ~20 missing)
Plus comprehensive cleaning for all 63 columns based on CSV audit report.
"""

import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import os


def clean_currency_value(value):
    """
    Clean currency values: remove $, commas, spaces and convert to float.
    Returns None for invalid/empty values (not 0.0 to preserve data integrity).
    """
    if pd.isna(value) or value == "" or value is None:
        return None
    
    try:
        # Handle numeric types directly
        if isinstance(value, (int, float)):
            return float(value) if value != 0 else None
        
        # Clean string representations
        cleaned = str(value).replace("$", "").replace(",", "").replace(" ", "").strip()
        if not cleaned or cleaned.lower() in ['nan', 'null', 'none', '']:
            return None
        return float(cleaned)
    except (ValueError, TypeError):
        return None


def standardize_boolean(value):
    """
    Convert various boolean representations to True/False.
    Handles: Yes/No, True/False, 1/0, Y/N, On/Off
    """
    if pd.isna(value) or value == "" or value is None:
        return False
    
    if isinstance(value, bool):
        return value
    
    str_val = str(value).lower().strip()
    return str_val in ["true", "yes", "1", "y", "on", "t"]


def clean_text_field(value, max_length=None):
    """Clean text fields: trim whitespace, handle nulls, enforce length limits."""
    if pd.isna(value) or value is None:
        return None
    
    result = str(value).strip()
    if not result or result.lower() in ['nan', 'null', 'none']:
        return None
    
    if max_length and len(result) > max_length:
        result = result[:max_length-3] + "..."
    
    return result


def clean_grants_dataset(
    input_path="data/grants/final_merged_enhanced.csv",
    output_path="data/cleaned/grants_cleaned.csv"
):
    """
    Main cleaning function that processes the grants dataset.
    """
    print("=" * 70)
    print("🧹 ImaraFund Grants Data Cleaning")
    print("=" * 70)
    print(f"📄 Input:  {input_path}")
    print(f"💾 Output: {output_path}")
    print("-" * 70)
    
    # Ensure input file exists
    input_file = Path(input_path)
    if not input_file.exists():
        raise FileNotFoundError(f"Grants CSV not found: {input_path}")
    
    # Load dataset with encoding fallback
    try:
        df = pd.read_csv(input_file, encoding='utf-8')
        print("✓ Loaded with UTF-8 encoding")
    except UnicodeDecodeError:
        try:
            df = pd.read_csv(input_file, encoding='latin-1')
            print("✓ Loaded with Latin-1 encoding")
        except Exception as e:
            raise Exception(f"Could not read CSV: {e}")
    
    print(f"✓ Loaded {len(df)} grants with {len(df.columns)} columns\n")
    
    # =================================================================
    # PRIMARY FIX: Fill data_source_url from website_url
    # =================================================================
    print("🔗 PRIMARY FIX: Filling data_source_url from website_url...")
    
    # Check if columns exist
    has_website = 'website_url' in df.columns
    has_data_source = 'data_source_url' in df.columns
    
    if not has_data_source:
        df['data_source_url'] = None
        print("  ℹ️  Created data_source_url column")
    
    if has_website:
        missing_before = df['data_source_url'].isna().sum()
        missing_website = df['website_url'].isna().sum()
        
        print(f"  📉 Missing data_source_url BEFORE: {missing_before} ({missing_before/len(df)*100:.1f}%)")
        print(f"  📉 Missing website_url: {missing_website} ({missing_website/len(df)*100:.1f}%)")
        
        # Fill using combine_first (preserves existing data_source_url values)
        df['data_source_url'] = df['data_source_url'].combine_first(df['website_url'])
        
        missing_after = df['data_source_url'].isna().sum()
        filled = missing_before - missing_after
        
        print(f"  ✅ Filled {filled} data_source_url values from website_url")
        print(f"  ✅ Missing data_source_url AFTER: {missing_after} ({missing_after/len(df)*100:.1f}%)")
    else:
        print("  ⚠️  website_url column not found")
    
    print()
    
    # =================================================================
    # SECONDARY FIXES: Clean dirty data types from audit report
    # =================================================================
    
    # 1. Clean currency/numeric columns (currently objects with $ and commas)
    print("💰 Cleaning currency columns...")
    currency_columns = [
        'estimated_value_amount', 'minimum_amount', 'maximum_amount',
        'minimum_revenue', 'maximum_revenue'
    ]
    
    for col in currency_columns:
        if col in df.columns:
            non_null_before = df[col].notna().sum()
            df[col] = df[col].apply(clean_currency_value)
            non_null_after = df[col].notna().sum()
            print(f"  ✓ {col}: {non_null_before} → {non_null_after} valid values")
    
    print()
    
    # 2. Standardize boolean columns (mixed string representations)
    print("✓ Standardizing boolean columns...")
    boolean_columns = [
        'repayment_required', 'environmental_focus', 'innovation_focus',
        'digital_focus', 'export_focus', 'women_focused', 'youth_focused',
        'agriculture_focused', 'green_climate_focused', 'technical_assistance',
        'mentorship_available', 'networking_opportunities', 'training_provided',
        'co_financing_required', 'co_financing_available', 'export_support',
        'technology_innovation', 'digital_application', 'verified'
    ]
    
    standardized_count = 0
    for col in boolean_columns:
        if col in df.columns:
            df[col] = df[col].apply(standardize_boolean)
            standardized_count += 1
    
    print(f"  ✓ Standardized {standardized_count} boolean columns to True/False")
    print()
    
    # 3. Clean integer columns (handle mixed types)
    print("🔢 Cleaning integer columns...")
    integer_columns = [
        'duration_months', 'minimum_employees', 'maximum_employees',
        'grace_period_months', 'total_beneficiaries', 'year_established'
    ]
    
    for col in integer_columns:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    
    print(f"  ✓ Validated {len([c for c in integer_columns if c in df.columns])} integer columns")
    print()
    
    # 4. Clean critical text columns
    print("📝 Cleaning text columns...")
    text_cleaning = {
        'program_name': 500,
        'institution_name': 500,
        'eligibility_criteria': None,
        'application_process': None,
        'notes': None,
        'special_features': None
    }
    
    for col, max_len in text_cleaning.items():
        if col in df.columns:
            df[col] = df[col].apply(lambda x: clean_text_field(x, max_len))
    
    print(f"  ✓ Cleaned {len([c for c in text_cleaning.keys() if c in df.columns])} text columns")
    print()
    
    # =================================================================
    # SAVE CLEANED DATA
    # =================================================================
    print("💾 Saving cleaned dataset...")
    
    # Create output directory
    output_file = Path(output_path)
    output_file.parent.mkdir(parents=True, exist_ok=True)
    
    # Save cleaned data
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"  ✓ Saved to: {output_path}")
    
    # Also save as 'latest' for easy reference
    latest_path = output_file.parent / "grants_cleaned_latest.csv"
    df.to_csv(latest_path, index=False, encoding='utf-8')
    print(f"  ✓ Latest version: {latest_path.name}")
    
    # =================================================================
    # GENERATE SUMMARY REPORT
    # =================================================================
    print("\n📊 Data Quality Summary:")
    print("-" * 70)
    
    # Missing data analysis
    missing_data = df.isna().sum()
    total_cells = len(df) * len(df.columns)
    filled_cells = df.notna().sum().sum()
    completeness = (filled_cells / total_cells) * 100
    
    print(f"  📈 Overall Completeness: {completeness:.1f}%")
    print(f"  📊 Total Records: {len(df):,}")
    print(f"  📋 Total Columns: {len(df.columns)}")
    
    # Key columns status
    key_columns = {
        'data_source_url': 'Data source (FIXED)',
        'website_url': 'Website contact',
        'estimated_value_amount': 'Funding amount',
        'target_sectors': 'Sector matching',
        'country': 'Geography matching',
        'geographic_scope': 'Geography matching'
    }
    
    print(f"\n  🎯 Key Matching Columns Status:")
    for col, desc in key_columns.items():
        if col in df.columns:
            missing = missing_data[col]
            missing_pct = (missing / len(df)) * 100
            status = "✅" if missing_pct < 10 else "✓" if missing_pct < 30 else "⚠️"
            print(f"    {status} {col}: {missing} missing ({missing_pct:.1f}%) - {desc}")
    
    print("\n" + "=" * 70)
    print("✅ DATA CLEANING COMPLETE!")
    print("=" * 70)
    print(f"\n📋 Summary of Changes:")
    print(f"  ✓ Fixed data_source_url (primary requirement)")
    print(f"  ✓ Cleaned currency columns (removed $, commas)")
    print(f"  ✓ Standardized {standardized_count} boolean columns")
    print(f"  ✓ Validated integer columns")
    print(f"  ✓ Cleaned text columns")
    
    print(f"\n📁 Output Files:")
    print(f"  • {output_file.name}")
    print(f"  • grants_cleaned_latest.csv (for easy access)")
    
    print(f"\n📋 Next Steps:")
    print(f"  1. Review the cleaned data in: {output_file.parent}")
    print(f"  2. Use grants_cleaned_latest.csv for backend migration")
    print(f"  3. Request Part 2 (10-step backend implementation)")
    
    print("\n" + "=" * 70 + "\n")
    
    return df, output_file


# =================================================================
# USAGE FUNCTIONS
# =================================================================

def run_from_notebook():
    """Convenience function for running from Control_V2.ipynb"""
    return clean_grants_dataset()


def run_with_custom_paths(input_csv, output_csv):
    """Run with custom file paths"""
    return clean_grants_dataset(input_csv, output_csv)


# =================================================================
# MAIN EXECUTION
# =================================================================

if __name__ == "__main__":
    # Run with default paths
    cleaned_df, output_path = clean_grants_dataset()
    print(f"🎉 Cleaning completed! Output saved to: {output_path}")


🧹 ImaraFund Grants Data Cleaning
📄 Input:  data/grants/final_merged_enhanced.csv
💾 Output: data/cleaned/grants_cleaned.csv
----------------------------------------------------------------------
✓ Loaded with UTF-8 encoding
✓ Loaded 103 grants with 63 columns

🔗 PRIMARY FIX: Filling data_source_url from website_url...
  📉 Missing data_source_url BEFORE: 83 (80.6%)
  📉 Missing website_url: 20 (19.4%)
  ✅ Filled 83 data_source_url values from website_url
  ✅ Missing data_source_url AFTER: 0 (0.0%)

💰 Cleaning currency columns...
  ✓ estimated_value_amount: 102 → 95 valid values
  ✓ minimum_amount: 20 → 20 valid values
  ✓ maximum_amount: 20 → 20 valid values
  ✓ minimum_revenue: 63 → 61 valid values
  ✓ maximum_revenue: 63 → 62 valid values

✓ Standardizing boolean columns...
  ✓ Standardized 19 boolean columns to True/False

🔢 Cleaning integer columns...
  ✓ Validated 6 integer columns

📝 Cleaning text columns...
  ✓ Cleaned 6 text columns

💾 Saving cleaned dataset...
  ✓ Saved to: data/